<a href="https://colab.research.google.com/github/ArshiAbolghasemi/AI-UT/blob/main/genetic_algorithms/genetic_algorithms/knapsack_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Knapsack Problem
In this project we're going to solve our knapsack problem using **Genetic Algorithm**

## Problem Description

Consider that we have a knapsack, and we want to go on a picnic. We have some snacks that we want to take with us. Each snack has a value, and we also have a certain amount of each snack that we can take as much as we want. It's obvious that if we take half of a snack, we will have half its value. Also there is some constraints:
- The sum of the values of our snacks shouldn't exceed a special amount that is given to us.
- The number of snacks that we pick should be within the interval that is given to us.
- The amount of the chosen snack shouldn't exceed the available amount of it.

The snack data is located in snacks.csv in the current directory.

# Settings And Imports

In [94]:
#imports
import os
import csv
import random
from functools import reduce
import sys
import numpy as np
from typing import List, Union

#settings
MAXIMUM_SNACKS_WEIGTHS=18
MINIMUM_SNACKS_VALUES=30
MINIMUM_SNACKS_COUNT=1
MAXIMUM_SNACKS_COUNT=10

POPULATION_SIZE=3000
GENERATIONS_COUNT=100

CROSSOVER_RATE=0.8
MUTATION_RATE=0.2
SNACK_CHOSEN_RATE=0.8

# Inputs
First, lets read input datas from snacks.csv file

In [3]:
CURRENT_DIR = os.getcwd()
SNACKS_CSV_FILE_PATH = os.path.join(CURRENT_DIR, 'snacks.csv')

snacks = []
snacks_available_weight = {}
snacks_values = {}
with open(SNACKS_CSV_FILE_PATH, mode='r') as snacks_file:
    snakcs_csv_reader = csv.reader(snacks_file)

    next(snakcs_csv_reader)

    for row in snakcs_csv_reader:
      snack = row[0]
      snacks.append(snack)
      snacks_available_weight[snack] = float(row[1])
      snacks_values[snack] = float(row[2])

# Gene And Chromosme
First, we need a definition for our gene and then using that creating a chromosome, that is a potential solution to our problem.

## Gene
In the context of this problem, each gene represents an item and the chosen amount. If we don't choose an item, its value is equal to zero; otherwise, it is equal to the amount chosen.

In [4]:
class Gene:

  def __init__(self, _snack_name: str, weight: float):
    self.snack_name = _snack_name
    self.weight = weight

  def get_weight(self) -> float:
    return self.weight

  def set_weight(self, _weight) -> None:
    self.weight = _weight;

  def get_value(self) -> float:
    return (
        (self.weight / float(snacks_available_weight[self.snack_name])) *
        float(snacks_values[self.snack_name])
    )

  def get_snack_name(self) -> str:
    return self.snack_name

## Chromosome
A chromosome is a collection of genes that represents a potential solution to the knapsack problem. It encodes a set of items to be included in the knapsack.

In [22]:
class Chromosome:

  def __init__(self, _genes: List[Gene]):
    self.genes = _genes

  def get_genes(self) -> List[Gene]:
    return self.genes

  def get_total_weight(self) -> float:
    return reduce(lambda acc, gene: acc + gene.get_weight(), self.genes, 0.0)

  def get_total_value(self) -> float:
    return reduce(lambda acc, gene: acc + gene.get_value(), self.genes, 0.0)

  def get_snacks_count(self) -> int:
    return len(list(filter(lambda gene: gene.get_weight() > 0.0, self.genes)))

# Generating Initial Population

In this step, we should generate a population of chromosomes stochastically. Counts of our populatin is defined in our [settings](#scrollTo=k6DVDoLt4kDT)

In [85]:
def generate_initial_population() -> List[Chromosome]:
  population = list()
  for _ in range(POPULATION_SIZE):
    genes = [
        Gene(snack,
             0.0 if random.choices([True, False], weights=[1 - SNACK_CHOSEN_RATE, SNACK_CHOSEN_RATE])[0]
             else random.uniform(0, float(snacks_available_weight[snack])))
        for snack in snacks
    ]

    population.append(Chromosome(genes))

  return population

# Fitness Function

Now, we need a fitness function to identify the better chromosome that models our problem. We define the fitness function as follows:

- If total value chromosome is less than minimum snack value or total weight chromosome is more than maximum sancks weight or snacks count is not snacks count interval, fitness is zero
- Otherwise, we consider the fitness score to default to one and then penalize it whenever it breaks constraints.

In [72]:
def fitness(chromosome: Chromosome) -> float:
  total_weight = chromosome.get_total_weight();
  total_value = chromosome.get_total_value();
  snacks_count = chromosome.get_snacks_count();

  fitness_score = 3.0
  if total_weight > float(MAXIMUM_SNACKS_WEIGTHS):
    fitness_score -= (
        (total_weight - MAXIMUM_SNACKS_WEIGTHS) /
        total_weight
        )

  if total_value < float(MINIMUM_SNACKS_VALUES):
    fitness_score -= (
        (MINIMUM_SNACKS_VALUES - total_value) /
        MINIMUM_SNACKS_VALUES
        )

  if not MINIMUM_SNACKS_COUNT <= snacks_count <= MAXIMUM_SNACKS_COUNT:
    fitness_score -= (
        min(abs(snacks_count - MINIMUM_SNACKS_COUNT),
            abs(snacks_count - MAXIMUM_SNACKS_COUNT)) /
             (len(snacks))
    )

  return fitness_score / 3.0

Fitness one means that we found a solution for the problem

# Crossover and Mutation
Selected chromosomes are used to create offspring through genetic operators such as crossover and mutation. Crossover involves exchanging genetic material between parent chromosomes to create new offspring chromosomes, while mutation introduces small random changes to the chromosomes to maintain diversity in the population.

## Crossover

In [8]:
def crossover(chromosome1: Chromosome, chromosome2: Chromosome):
  genes1 = chromosome1.get_genes()
  genes2 = chromosome2.get_genes()
  i = random.randint(0, len(snacks))
  return Chromosome(genes1[:i] + genes2[i:]), Chromosome(genes2[:i] + genes1[i:])

## Mutation

In [86]:
def mutation(chromosome: Chromosome, mutation_rate: float) -> Chromosome:
  genes = chromosome.get_genes()
  for gene in genes:
    if random.choices([True, False],
                      weights=[1 - mutation_rate, mutation_rate])[0]:
      continue

    mutation_point = np.random.randint(len(genes))
    genes[mutation_point].set_weight(random.uniform(
              0, float(snacks_available_weight[gene.get_snack_name()])))

  return chromosome

# Selection And Evolution

Now, using the fitness function, we need to approach the selection of chromosomes, and then, by employing crossover and mutation, create a new generation.

## Selection

We use **roulette wheel selection** that used fitness socre to select parents.
First, calculate the fitness values for each individual in the population based on their performance on the fitness function. Then, Normalize the fitness values to ensure that they sum up to 1. This step ensures that the probabilities of selection are proportional to the individuals' fitness values.Imagine a roulette wheel where each individual's portion of the wheel is proportional to its fitness value. This means that individuals with higher fitness values occupy larger portions of the wheel, making them more likely to be selected

In [10]:
def roulette_wheel_selection(population, fitness_values):
    total_fitness = sum(fitness_values)
    selection_probabilities = [fitness / total_fitness for fitness in fitness_values]

    return random.choices(population, weights=selection_probabilities, k=2)

## Evolotion

Using roulette wheel selection, for each generation, we select two parents. With a probability of p<sub>c</sub>, we apply crossover; otherwise, we keep them. Then, with a probability of p<sub>m</sub>, we apply mutation to them and add the resulting chromosome to the next generation. We repeat this process
for population_size time.

In [87]:
def evolution(population, p_c, p_m) -> List[Chromosome]:
  fitness_values = [fitness(chromosome) for chromosome in population]

  new_population = list()

  for _ in range(POPULATION_SIZE // 2):
    parent1, parent2 = roulette_wheel_selection(population, fitness_values)
    if random.choices([True, False], weights=[p_c, 1 - p_c])[0]:
      offspring1, offspring2 = crossover(parent1, parent2)
    else:
      offspring1, offspring2 = parent1, parent2

    offsprings = [mutation(offspring, p_m) for offspring in [offspring1, offspring2]]
    new_population.extend(offsprings)

  return new_population

# Genetic Algorithm
Now using above methods, let's implemnt genetic algorithm. Iterate over generation, apply evolotion on them, if we have chromosme with fitness one, then we find a solution otherwise we continue this process until find a solution

In [90]:
def genetic_algorithm() -> Union[Chromosome, None]:
  population = generate_initial_population()

  max_fitness_chromosome = None
  max_fitness_score = float('-inf')
  for _ in range(GENERATIONS_COUNT):
    for chromosome in population:
      fitness_score = fitness(chromosome)
      if fitness_score == 1.0:
          return chromosome
      if fitness_score > max_fitness_score:
        max_fitness_chromosome = chromosome
        max_fitness_score = fitness_score

    population = evolution(population, CROSSOVER_RATE, MUTATION_RATE)

  return max_fitness_chromosome

Now, let's solve problem

In [ ]:
def knapsack() -> None:
  chromosome = genetic_algorithm()

  if chromosome is None:
    print("Solution Not Found!")
  else:
    for gene in chromosome.get_genes():
      print(gene.get_snack_name() + ': ' + str(gene.get_weight()) + '\n')
    print('Total Weight: ' + str(chromosome.get_total_weight()) + '\n')
    print('Total Value: ' + str(chromosome.get_total_value()) + '\n')
    print('Fitenss: ' + str(fitness(chromosome)))

knapsack()

# Qeustions



1.   What problems does too little or too much initial population size cause?

> If we have too small an initial population, we will have less diversity, and the only diversity we have is generated by mutation. Therefore, if our population does not move in the proper direction, we may reach the answer very late. On the other hand, too much initial population size increases memory usage and execution time.


2. If the population size increases in each iteration, what effect does it have on the accuracy and speed of the algorithm?

> It depends on implementation, if we implement and use it properly, then we have more diversity that helps crossover to have a better result. On the other hand it may cuase redundant computations and increases execution time

3. Compare the effect of each operation of crossover and mutation. Can only one of them used? Why?

> - Crossover:
Crossover is responsible for combining genetic material from two parent individuals to create new offspring.
It promotes exploration of the search space by exchanging information between individuals.
It helps in preserving good solutions found so far and can lead to the creation of new, potentially better solutions.
- Mutation:
Mutation introduces random changes to individual chromosomes, promoting genetic diversity.
It helps in escaping local optima by exploring new areas of the search space.
It can prevent premature convergence by introducing small, random changes to individual solutions.
While both crossover and mutation contribute to the effectiveness of genetic algorithms, it is theoretically possible to use only one of them. However, doing so may result in suboptimal performance or failure to find optimal solutions, depending on the characteristics of the problem being solved.
<br>
Using only crossover may lead to premature convergence if the initial population lacks diversity, as there are no mechanisms to introduce new genetic material.
<br>
Using only mutation may result in slow convergence or difficulty in finding good solutions, as there is no mechanism for combining information from different individuals.

4. In your opinion, what solutions are available to obtain the answer to this particular problem more quickly?

> Change selection approach, for example use elitisim to prevent loose few good solutions, also change fitness function to consider parameters like boundary for snacks weight and value and count to check that we can improve this chromosome or not

5. Despite the use of these methods, it is still possible that the chromosomes don't change after a few more steps. Explain the reason for this and the problems it causes. What do you suggest to solve it?

> It may because don't implement crossover or mutation correctly. Also, it may have happened because of our selection that we only have high-ranked chromosomes and so we lose our diversity after a few steps. For solving this problem one of our approaches is to select the chromosomes that the lower-ranked chromosomes are in the population to7. prevent losing our diversity.

6. What solution do you suggest to end the program if the problem has no answer?

> First, we should have a time limit or iteration limit for our algorithm to be sure that our algorithm is halted. then we could return the most fitted chromosome in our population as an optimal chromosome.












